# 5.7 LCEL Based Memory

#### 학습목표: LangChain Expression 언어를 이용하여 생성된 체인에 메모리를 추가하는 방법 배우기

In [15]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

chain = prompt | llm

print(memory.load_memory_variables({})) # 이 출력의 결과는 {'chat_history': []}라는 객체 형태이다.
# 여기서 chat_history는 우리가 이전에 넣어줬던 memory_key이자 객체의 키가 된다.
# 따라서 "memory.load_memory_variables({})["chat_history"]"라고 하게 되면
# 이 객체중에 'chat_history'라고 하는 키의 '값'에 접근할 수 있다.
# 이 '값'은 리스트형태인데, 여기에는 지금까지의 대화 내용이 객체형태(또는 클래스형태)로 저장되어 있다.


chain.invoke({
    "chat_history": memory.load_memory_variables({})["chat_history"],
    "question": "My name is Nico"
})

{'chat_history': []}


AIMessage(content='Hello Nico! How can I assist you today?')

그런데 이 접근방식의 문제는 우리가 chain을 호출할 때마다 chat_history도 추가해줘야 한다는 것이다.

즉, 체인을 호출 할 때마다 다음처럼 해줘야 한다.

```python
chain.invoke({
    "chat_history": memory.load_memory_variables({})["chat_history"],
    "question": "How are you?"
})

chain.invoke({
    "chat_history": memory.load_memory_variables({})["chat_history"],
    "question": "I'm happy!"
})
```

이 방법도 좋지만, 더 좋은 방법이 있다. (다음 코드에서 설명)

In [16]:
from langchain.schema.runnable import RunnablePassthrough

def load_memory(_): # 여기서 괄호 안에 (_) 언더스크롤을 넣어준 이유는 들어오는 인풋값을 무시해도 된다는 말이다.
    # 그냥 비워놓으면 되지 왜 굳이 언더스크롤을 써야하나? 라고 생각할 수 있다.
    # 언더스크롤을 써야 하는 이유는 체인에 있는 모든 컴포넌트(여기에선 RunnablePassthrough.assign, prompt, llm)
    # 들은 input을 받을거고, 또 output을 주게된다. 즉, input값이 자동적으로 주어지게 된다는 말이다.
    # def load_memory(input): print(input) 해보면 알게 된다.
    # 여기서는 인풋값이 "question": "My name is Nico" 로 주어진다. 이 값을 무시하라는 의미에서 언더스크롤을 쓰는것이다.
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context({"input":question}, {"output":result.content})
    print(result)


#### <load_memory 함수의 언더스코어(_) 파라미터 설명>

##### 1. 파이프라인의 데이터 흐름
```python
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm
```
이 파이프라인에서:
- RunnablePassthrough.assign은 입력 데이터를 받음
- 이 입력 데이터는 자동으로 load_memory 함수에 전달됨
- 실제 입력 데이터 예: `{"question": "My name is Nico"}`

##### 2. 언더스코어(_) 사용 이유
- Python에서 언더스코어는 "이 값을 사용하지 않겠다"는 관례적 표현
- 함수가 파라미터를 받아야 하지만 실제로는 사용하지 않을 때 사용
- 코드 리뷰어에게 "이 파라미터는 의도적으로 무시됨"을 명시적으로 보여줌

##### 3. 실제 동작
```python
def load_memory(_):  # {"question": "My name is Nico"} 가 _ 로 들어옴
    return memory.load_memory_variables({})["chat_history"]  # 이 값만 사용
```

##### 4. 빈 파라미터와의 차이점
```python
def load_memory():  # ❌ 에러 발생
    return memory.load_memory_variables({})["chat_history"]
```
- 빈 파라미터로 정의하면 파이프라인에서 자동으로 전달되는 입력값을 받을 수 없음
- 파이프라인의 규칙: 모든 컴포넌트는 입력을 받을 수 있어야 함


In [17]:
invoke_chain("My name is Nico")

content='Nice to meet you, Nico! How can I assist you today?'


In [18]:
invoke_chain("What is my name?")

content='Your name is Nico.'


In [19]:
invoke_chain("I live in Seoul")

content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'


In [20]:
invoke_chain("I love football")

content="That's great to hear! Football is a popular sport enjoyed by many around the world. Do you have a favorite team or player?"


In [21]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content="The human introduces themselves as Nico. The AI responds by saying it's nice to meet Nico and asks how it can assist them today."),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Nico.'),
  HumanMessage(content='I live in Seoul'),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'),
  HumanMessage(content='I love football'),
  AIMessage(content="That's great to hear! Football is a popular sport enjoyed by many around the world. Do you have a favorite team or player?")]}

### RunnablePassthrough와 assign

#### RunnablePassthrough란?
LCEL(LangChain Expression Language)에서 제공하는 유틸리티 클래스로, 입력 데이터를 수정하거나 확장할 수 있게 해줍니다.

#### 주요 기능

##### 1. assign() 메서드
- 입력 데이터에 새로운 키-값 쌍을 추가
- 기존 입력은 그대로 유지하면서 새로운 데이터만 추가
- 파이프라인의 다음 단계로 확장된 데이터를 전달

##### 2. 사용 예시
```python
from langchain.schema.runnable import RunnablePassthrough

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

# 입력
chain.invoke({
    "question": "My name is Nico"
})

# 실제 처리되는 데이터
{
    "question": "My name is Nico",
    "chat_history": [메모리에서 로드된 대화 내용]
}
```

---------------------------------------
#### 이전 방식과의 비교

##### 이전 방식 (수동 추가)
```python
chain.invoke({
    "chat_history": memory.load_memory_variables({})["chat_history"],
    "question": "My name is Nico"
})
```

##### RunnablePassthrough 사용 (자동 추가)
```python
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm
chain.invoke({"question": "My name is Nico"})
```

#### 장점
1. 코드 간소화
2. 자동화된 데이터 확장
3. 파이프라인 구성의 명확성
4. 재사용 가능한 구조
